## 📊 IMAGO Revenue Pipeline Analysis

# Revenue Data Quality & Attribution Analysis

This notebook walks through:

1. Loading and profiling the data  
2. Identifying positions tied to unpaid invoices  
3. Quantifying revenue on placeholder media (`Bildnummer = 100000000`)  
4. Finding invoices with no line-items  

---


In [56]:
# Cell 2 (updated): Load the data from the root directory
import pandas as pd
import os
print("Files in cwd:", os.listdir('.'))

invoices = pd.read_csv(
    'invoices.csv',
    sep=';',
    decimal=',',
    thousands='.',
    parse_dates=['ReDatum', 'Zahlungsdatum']
)
positions = pd.read_csv(
    'positions.csv',
    sep=';',
    decimal=',',
    thousands='.',
    parse_dates=['VerDatum']
)
customers = pd.read_csv(
    'customers.csv',
    sep=';'
)

# Quick shape check
print("🧾 Invoices:", invoices.shape)
print("📄 Positions:", positions.shape)
print("👥 Customers:", customers.shape)


Files in cwd: ['invoices.csv', 'customers.csv', 'simplified_schema_mssql.txt', 'positions.csv']
🧾 Invoices: (2000, 8)
📄 Positions: (129092, 6)
👥 Customers: (601, 4)


In [57]:
# Cell 3: Peek at each table
display(invoices.head(3))
display(positions.head(3))
display(customers.head(3))


,ReNummer,SummeNetto,MwStSatz,ZahlungsbetragBrutto,KdNr,Summenebenkosten,ReDatum,Zahlungsdatum
0,103601,1178670,0,0.0,78911,0.0,2025-04-04,NaT
1,103600,37900,7,0.0,79666,0.0,2025-04-03,NaT
2,103599,16000,7,0.0,30145,0.0,2025-04-04,NaT


,id,ReId,KdNr,Nettobetrag,Bildnummer,VerDatum
0,4154495,102038,50292.0,1900.0,168161043.0,2022-09-29
1,4154496,102038,50292.0,1900.0,108514408.0,2022-09-12
2,4154497,102038,50292.0,1900.0,55374589.0,2022-09-08


,id,Kdnr,Verlagsname,Region
0,5,20172,1. FC Nürnberg,Nürnberg
1,19,20137,Allgäuer Zeitung / Allgäuer Zeitungsverlag GmbH,Bodensee
2,27,20115,Augsburger Allgemeine,München


### 1. Positions linked to invoices missing payment info  
We join `positions.ReId → invoices.ReNummer` and count where `Zahlungsdatum` is null.


In [58]:
# Cell 4: Unpaid positions count & total
pos_inv = (
    positions
    .merge(
        invoices[['ReNummer', 'Zahlungsdatum']],
        left_on='ReId',
        right_on='ReNummer',
        how='left'
    )
)
unpaid = pos_inv[pos_inv['Zahlungsdatum'].isna()]

count_unpaid = len(unpaid)
total_unpaid = unpaid['Nettobetrag'].sum()

print(f"Number of positions on unpaid invoices: {count_unpaid}")
print(f"Total Nettobetrag of those positions: €{total_unpaid:,.2f}")


Number of positions on unpaid invoices: 18011
Total Nettobetrag of those positions: €57,663,120.00


### 2. Revenue attributed to placeholder media ID (`Bildnummer = 100000000`)
Filter `positions` for the placeholder and sum `Nettobetrag`.


In [59]:
# Cell 5: Placeholder-media revenue
placeholder = positions[positions['Bildnummer'] == 100000000]
count_ph = len(placeholder)
total_ph = placeholder['Nettobetrag'].sum()

print(f"Count of placeholder positions: {count_ph}")
print(f"Total Nettobetrag on placeholder media: €{total_ph:,.2f}")


Count of placeholder positions: 2407
Total Nettobetrag on placeholder media: €131,989,791.00


### 3. Invoices with no positions attached  
Find all `ReNummer` in invoices not present in `positions.ReId`.


In [60]:
# Cell 6: Orphan invoices
invoices_with_pos = positions['ReId'].unique()
orphan = invoices[~invoices['ReNummer'].isin(invoices_with_pos)]

count_orphan = len(orphan)
print(f"Number of invoices with zero positions: {count_orphan}")
display(orphan[['ReNummer', 'ReDatum', 'SummeNetto']].head())


Number of invoices with zero positions: 2


,ReNummer,ReDatum,SummeNetto
1283,102318,2025-01-03,31600
1626,101975,2024-12-03,20000


In [68]:
# Cell 7: Render the Findings Summary dynamically
from IPython.display import Markdown, display

summary = f"""
## 📝 Findings Summary

1. **Unpaid Positions**  
   {count_unpaid:,} positions on invoices without a `Zahlungsdatum`.  
   €{total_unpaid:,.2f} total net amount.

2. **Placeholder Media ID**  
   {count_ph:,} line-items booked to `Bildnummer = 100000000`.  
   €{total_ph:,.2f} total net revenue on these.

3. **Orphan Invoices**  
   {count_orphan:,} invoices have no associated positions.  
"""

display(Markdown(summary))



## 📝 Findings Summary

1. **Unpaid Positions**  
   18,011 positions on invoices without a `Zahlungsdatum`.  
   €57,663,120.00 total net amount.

2. **Placeholder Media ID**  
   2,407 line-items booked to `Bildnummer = 100000000`.  
   €131,989,791.00 total net revenue on these.

3. **Orphan Invoices**  
   2 invoices have no associated positions.  


> **Next Steps**  
> - Enforce FK relationships at ingestion.  
> - Reject or flag `Bildnummer=100000000`.  
> - Automate alerts when unpaid-position totals or orphan-invoice counts exceed thresholds.  